<a href="https://colab.research.google.com/github/Jesica3018/APMKelompok6/blob/main/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import warnings
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('BreadBasket.csv')
data.head()

Date      Time  Transaction           Item
0  2016-10-30  09:58:11            1          Bread
1  2016-10-30  10:05:34            2   Scandinavian
2  2016-10-30  10:05:34            2   Scandinavian
3  2016-10-30  10:07:57            3  Hot chocolate
4  2016-10-30  10:07:57            3            Jam

In [3]:
# Exploring the columns of the data
data.columns

Index(['Date', 'Time', 'Transaction', 'Item'], dtype='object')

In [4]:
# Exploring the different Item of transactions
data.Item.unique()

array(['Bread', 'Scandinavian', 'Hot chocolate', 'Jam', 'Cookies',
       'Muffin', 'Coffee', 'Pastry', 'Medialuna', 'Tea', 'NONE',
       'Tartine', 'Basket', 'Mineral water', 'Farm House', 'Fudge',
       'Juice', "Ella's Kitchen Pouches", 'Victorian Sponge', 'Frittata',
       'Hearty & Seasonal', 'Soup', 'Pick and Mix Bowls', 'Smoothies',
       'Cake', 'Mighty Protein', 'Chicken sand', 'Coke',
       'My-5 Fruit Shoot', 'Focaccia', 'Sandwich', 'Alfajores', 'Eggs',
       'Brownie', 'Dulce de Leche', 'Honey', 'The BART', 'Granola',
       'Fairy Doors', 'Empanadas', 'Keeping It Local', 'Art Tray',
       'Bowl Nic Pitt', 'Bread Pudding', 'Adjustment', 'Truffles',
       'Chimichurri Oil', 'Bacon', 'Spread', 'Kids biscuit', 'Siblings',
       'Caramel bites', 'Jammie Dodgers', 'Tiffin', 'Olum & polenta',
       'Polenta', 'The Nomad', 'Hack the stack', 'Bakewell',
       'Lemon and coconut', 'Toast', 'Scone', 'Crepes', 'Vegan mincepie',
       'Bare Popcorn', 'Muesli', 'Crisps', 'Pi

In [5]:
#Droping all none values
data = data.drop(data[data.Item == 'none'].index)

In [6]:
data['datetime'] = pd.to_datetime(data['Date']+" "+data['Time'])
data['Week'] = data['datetime'].dt.week
data['Month'] = data['datetime'].dt.month
data['Weekday'] = data['datetime'].dt.weekday
data['Hours'] = data['datetime'].dt.hour

In [7]:
df1=data[['Date','Transaction', 'Month','Week', 'Weekday','Hours']]

In [8]:
sns.countplot(x='Weekday',data=df1)

In [9]:
sns.countplot(x='Hours',data=df1)

In [10]:
hot_encoded_df = data.groupby(['Transaction', 'Item'])['Item'].count().unstack().reset_index().fillna(0).set_index('Transaction')

In [11]:
hot_encoded_df.head()

Item         Adjustment  Afternoon with the baker  Alfajores  Argentina Night  \
Transaction                                                                     
1                   0.0                       0.0        0.0              0.0   
2                   0.0                       0.0        0.0              0.0   
3                   0.0                       0.0        0.0              0.0   
4                   0.0                       0.0        0.0              0.0   
5                   0.0                       0.0        0.0              0.0   

Item         Art Tray  Bacon  Baguette  Bakewell  Bare Popcorn  Basket  ...  \
Transaction                                                             ...   
1                 0.0    0.0       0.0       0.0           0.0     0.0  ...   
2                 0.0    0.0       0.0       0.0           0.0     0.0  ...   
3                 0.0    0.0       0.0       0.0           0.0     0.0  ...   
4                 0.0    0.0       0.0       0.0           0.0     0.0  ...   
5                 0.0    0.0       0.0       0.0           0.0     0.0  ...   

Item         The BART  The Nomad  Tiffin  Toast  Truffles  Tshirt  \
Transaction                                                         
1                 0.0        0.0     0.0    0.0       0.0     0.0   
2                 0.0        0.0     0.0    0.0       0.0     0.0   
3                 0.0        0.0     0.0    0.0       0.0     0.0   
4                 0.0        0.0     0.0    0.0       0.0     0.0   
5                 0.0        0.0     0.0    0.0       0.0     0.0   

Item         Valentine's card  Vegan Feast  Vegan mincepie  Victorian Sponge  
Transaction                                                                   
1                         0.0          0.0             0.0               0.0  
2                         0.0          0.0             0.0               0.0  
3                         0.0          0.0             0.0               0.0  
4                         0.0          0.0             0.0               0.0  
5                         0.0          0.0             0.0               0.0  

[5 rows x 95 columns]

In [12]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
hot_encoded_df = hot_encoded_df.applymap(encode_units)

In [15]:
frequent_itemsets = apriori(hot_encoded_df, min_support=0.01, use_colnames=True)

In [16]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules.head(10)

antecedents      consequents  antecedent support  consequent support  \
0         (Coffee)      (Alfajores)            0.475081            0.036093   
1      (Alfajores)         (Coffee)            0.036093            0.475081   
2         (Pastry)          (Bread)            0.085510            0.324940   
3          (Bread)         (Pastry)            0.324940            0.085510   
4        (Brownie)         (Coffee)            0.039765            0.475081   
5         (Coffee)        (Brownie)            0.475081            0.039765   
6           (Cake)         (Coffee)            0.103137            0.475081   
7         (Coffee)           (Cake)            0.475081            0.103137   
8           (Cake)  (Hot chocolate)            0.103137            0.057916   
9  (Hot chocolate)           (Cake)            0.057916            0.103137   

    support  confidence      lift  leverage  conviction  
0  0.019515    0.041078  1.138116  0.002368    1.005199  
1  0.019515    0.540698  1.138116  0.002368    1.142861  
2  0.028958    0.338650  1.042194  0.001172    1.020731  
3  0.028958    0.089119  1.042194  0.001172    1.003961  
4  0.019515    0.490765  1.033013  0.000624    1.030799  
5  0.019515    0.041078  1.033013  0.000624    1.001369  
6  0.054349    0.526958  1.109196  0.005350    1.109667  
7  0.054349    0.114399  1.109196  0.005350    1.012717  
8  0.011331    0.109868  1.897010  0.005358    1.058364  
9  0.011331    0.195652  1.897010  0.005358    1.115019

In [17]:
rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.5)]

antecedents consequents  antecedent support  consequent support  \
1        (Alfajores)    (Coffee)            0.036093            0.475081   
6             (Cake)    (Coffee)            0.103137            0.475081   
12         (Cookies)    (Coffee)            0.054034            0.475081   
15   (Hot chocolate)    (Coffee)            0.057916            0.475081   
17           (Juice)    (Coffee)            0.038296            0.475081   
19       (Medialuna)    (Coffee)            0.061379            0.475081   
23            (NONE)    (Coffee)            0.079005            0.475081   
24          (Pastry)    (Coffee)            0.085510            0.475081   
27        (Sandwich)    (Coffee)            0.071346            0.475081   
28           (Scone)    (Coffee)            0.034309            0.475081   
31  (Spanish Brunch)    (Coffee)            0.018046            0.475081   
33           (Toast)    (Coffee)            0.033365            0.475081   

     support  confidence      lift  leverage  conviction  
1   0.019515    0.540698  1.138116  0.002368    1.142861  
6   0.054349    0.526958  1.109196  0.005350    1.109667  
12  0.028014    0.518447  1.091280  0.002343    1.090053  
15  0.029378    0.507246  1.067704  0.001863    1.065276  
17  0.020460    0.534247  1.124537  0.002266    1.127031  
19  0.034939    0.569231  1.198175  0.005779    1.218561  
23  0.042073    0.532537  1.120938  0.004539    1.122908  
24  0.047214    0.552147  1.162216  0.006590    1.172079  
27  0.037981    0.532353  1.120551  0.004086    1.122468  
28  0.017941    0.522936  1.100729  0.001642    1.100310  
31  0.010807    0.598837  1.260494  0.002233    1.308493  
33  0.023502    0.704403  1.482699  0.007651    1.775789